<a href="https://colab.research.google.com/github/mb16biswas/llm/blob/main/ensemble_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets==2.16.1
!pip install scikit-learn numpy pandas
!pip install transformers==4.30
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git
!pip install -q bitsandbytes einops sentencepiece
!pip install evaluate seqeval
!pip uninstall -y apex
!pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.10.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; p

In [30]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Dict
import math
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import  get_linear_schedule_with_warmup
from tqdm import tqdm
import numpy as np
from torch.nn.utils.rnn import pad_sequence

In [3]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
base_model = T5ForConditionalGeneration.from_pretrained('t5-base')
semantic_encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = base_model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
# class ConfidenceAwareFusionModel(nn.Module):
#     def __init__(self, encoder_decoder_model_name="google-t5/t5-small", sentence_encoder_name="all-mpnet-base-v2"):
#         super().__init__()
#         self.tokenizer = T5Tokenizer.from_pretrained(encoder_decoder_model_name)
#         self.model = T5ForConditionalGeneration.from_pretrained(encoder_decoder_model_name)
#         self.sentence_encoder = SentenceTransformer(sentence_encoder_name)

#         # Confidence projection for encoder token fusion
#         self.conf_proj = nn.Linear(1, self.model.config.d_model)

#         # Optional: for decoder attention bias (not injected here but can be extended)
#         self.bias_mlp = nn.Sequential(
#             nn.Linear(1, self.model.config.d_model),
#             nn.Tanh(),
#             nn.Linear(self.model.config.d_model, 1)
#         )


class ConfidenceAwareFusionModel(nn.Module):
    def __init__(self, encoder_decoder_model_name="google-t5/t5-small", sentence_encoder_name="all-mpnet-base-v2"):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(encoder_decoder_model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(encoder_decoder_model_name)
        self.sentence_encoder = SentenceTransformer(sentence_encoder_name)

        hidden_size = self.model.config.hidden_size
        self.conf_proj = nn.Linear(1, hidden_size)
        self.bias_mlp = nn.Sequential(
            nn.Linear(1, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )

    def compute_semantic_agreement(self, answers: List[str]) -> torch.Tensor:
        embeddings = self.sentence_encoder.encode(answers, convert_to_tensor=True)  # (n, d)
        sim_matrix = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)  # (n, n)
        return sim_matrix

    # def compute_final_confidence_scores(self, raw_confidences: List[float], sem_sim_matrix: torch.Tensor, lambda_param=0.7) -> List[float]:
    #     n = len(raw_confidences)
    #     norm_conf = torch.tensor(raw_confidences) / max(raw_confidences)
    #     final_scores = []

    #     for i in range(n):
    #         sem_agree = (torch.sum(sem_sim_matrix[i]) - 1) / (n - 1)
    #         score = lambda_param * norm_conf[i] + (1 - lambda_param) * sem_agree
    #         final_scores.append(score.item())

    #     return final_scores  # List[float]

    def compute_final_confidence_scores(self, raw_confidences: List[float], sem_sim_matrix: torch.Tensor, lambda_param=0.7) -> List[float]:
        n = len(raw_confidences)
        norm_conf = torch.tensor(raw_confidences) / max(raw_confidences)
        final_scores = []

        for i in range(n):
            if n == 1:
                sem_agree = 1.0  # or 0.0, or just use norm_conf[i]
            else:
                sem_agree = (torch.sum(sem_sim_matrix[i]) - 1) / (n - 1)
            score = lambda_param * norm_conf[i] + (1 - lambda_param) * sem_agree
            final_scores.append(score.item())

        return final_scores

    # def encode_with_confidence(self, input_ids_list, attention_masks, scores):
    #     all_embeddings, all_masks = [], []
    #     for input_ids, attn_mask, ai in zip(input_ids_list, attention_masks, scores):
    #         token_embeddings = self.model.encoder.embed_tokens(input_ids)  # (1, seq_len, dim)
    #         conf_vec = self.conf_proj(torch.tensor([[ai]], device=token_embeddings.device))  # (1, 1, dim)
    #         conf_vec = conf_vec.expand_as(token_embeddings)
    #         fused = token_embeddings + conf_vec
    #         all_embeddings.append(fused)
    #         all_masks.append(attn_mask)

    #     return torch.cat(all_embeddings, dim=1), torch.cat(all_masks, dim=1)


    def encode_with_confidence(self, input_ids_list, attention_masks, scores):
        all_embeddings, all_masks = [], []
        for input_ids, attn_mask, ai in zip(input_ids_list, attention_masks, scores):
            token_embeddings = self.model.encoder.embed_tokens(input_ids)  # (1, seq_len, dim)
            conf_vec = self.conf_proj(torch.tensor([[ai]], device=token_embeddings.device))  # (1, 1, dim)
            conf_vec = conf_vec.expand_as(token_embeddings)
            fused = token_embeddings + conf_vec
            all_embeddings.append(fused)
            all_masks.append(attn_mask)

        embeddings = [x.squeeze(0) for x in all_embeddings]  # [seq_len, hidden_dim]
        masks = [x.squeeze(0) for x in all_masks]            # [seq_len]

        # Pad
        padded_embeddings = pad_sequence(embeddings, batch_first=True)  # [num_segments, max_seq_len, hidden_dim]
        padded_masks = pad_sequence(masks, batch_first=True)            # [num_segments, max_seq_len]

        # Restore batch dim
        padded_embeddings = padded_embeddings.unsqueeze(0)  # [1, num_segments * max_seq_len, hidden_dim]
        padded_masks = padded_masks.unsqueeze(0)            # [1, num_segments * max_seq_len]

        # Finally, flatten across segments
        final_embeddings = padded_embeddings.view(1, -1, padded_embeddings.size(-1))  # [1, total_tokens, hidden_dim]
        final_masks = padded_masks.view(1, -1)                                        # [1, total_tokens]

        return final_embeddings, final_masks



    def forward(self, input_ids_list, attention_masks, raw_confidences, decoder_input_ids, labels=None):
        # Convert LLM outputs to decoded strings
        answers = [self.tokenizer.decode(ids[0], skip_special_tokens=True) for ids in input_ids_list]

        sem_sim = self.compute_semantic_agreement(answers)
        final_scores = self.compute_final_confidence_scores(raw_confidences, sem_sim)

        # Encoder
        fused_embeddings, fused_mask = self.encode_with_confidence(input_ids_list, attention_masks, final_scores)

        output = self.model(
            inputs_embeds=fused_embeddings,
            attention_mask=fused_mask,
            decoder_input_ids=decoder_input_ids,
            labels=labels,
            return_dict=True
        )

        return output



    def generate(self, input_ids_list, attention_masks, raw_confidences, max_length=128):
        answers = [self.tokenizer.decode(ids[0], skip_special_tokens=True) for ids in input_ids_list]
        sem_sim = self.compute_semantic_agreement(answers)
        final_scores = self.compute_final_confidence_scores(raw_confidences, sem_sim)

        fused_embeddings, fused_mask = self.encode_with_confidence(input_ids_list, attention_masks, final_scores)

        generated_ids = self.model.generate(
            inputs_embeds=fused_embeddings,
            attention_mask=fused_mask,
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )
        return self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [10]:
fusion_model = ConfidenceAwareFusionModel().to(device)

In [51]:
device

device(type='cpu')

In [52]:
fusion_model

ConfidenceAwareFusionModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Lin

In [32]:
fusion_model = ConfidenceAwareFusionModel().to(device)


# llm_outputs = [
#     "The Eiffel Tower is in Paris.",
#     "Paris hosts the Eiffel Tower.",
#     "Eiffel Tower is located in Paris, France."
# ]

llm_outputs =  [
            "The capital of Japan is Tokyo.",
            "Tokyo is the capital of Japan.",
            "Japan’s capital city is Mumbai."
        ]

confidences = [0.7, 0.6, 0.2]

# Tokenize all answers
input_ids_list, attn_masks = [], []
for out in llm_outputs:
    encoded = fusion_model.tokenizer(out, return_tensors="pt",padding=True, truncation=True).to(device)
    input_ids_list.append(encoded['input_ids'])
    attn_masks.append(encoded['attention_mask'])

# Training mode
target = "The Eiffel Tower is located in Paris."
# target = "The capital of Japan is Tokyo."
labels = fusion_model.tokenizer(target, return_tensors="pt",padding=True, truncation=True).input_ids.to(device)
decoder_input_ids = fusion_model.tokenizer("summarize:", return_tensors="pt").input_ids.to(device)

# output = fusion_model(input_ids_list, attn_masks, confidences, decoder_input_ids, labels=labels)
output = fusion_model(input_ids_list, attn_masks, confidences, decoder_input_ids=None, labels=labels)
print("Training loss:", output.loss.item())

# # Inference
fused_answer = fusion_model.generate(input_ids_list, attn_masks, confidences)
print("Fused Answer:", fused_answer)

Training loss: 2.680933952331543
Fused Answer: Tokyo ist die Hauptstadt Japans.


In [33]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Custom Dataset
class FusionDataset(Dataset):
    def __init__(self, all_outputs, all_confidences, all_targets, tokenizer):
        self.all_outputs = all_outputs
        self.all_confidences = all_confidences
        self.all_targets = all_targets
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.all_outputs)

    def __getitem__(self, idx):
        llm_outputs = self.all_outputs[idx]
        confidences = self.all_confidences[idx]
        target = self.all_targets[idx]

        # Tokenize input answers
        input_ids_list, attention_masks = [], []
        for out in llm_outputs:
            encoded = self.tokenizer(out, return_tensors="pt", padding=True, truncation=True)
            input_ids_list.append(encoded['input_ids'].squeeze(0))  # (seq_len,)
            attention_masks.append(encoded['attention_mask'].squeeze(0))

        # Tokenize target
        label_ids = self.tokenizer(target, return_tensors="pt", padding=True, truncation=True).input_ids.squeeze(0)

        return input_ids_list, attention_masks, confidences, label_ids

# Instantiate model and optimizer
model = ConfidenceAwareFusionModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


from torch.nn.utils.rnn import pad_sequence

def fusion_collate_fn(batch):
    input_ids_list_batch = []
    attn_masks_batch = []
    confidences_batch = []
    labels_batch = []

    for input_ids_list, attn_masks_list, confs, label in batch:
        input_ids_list_batch.append(input_ids_list)
        attn_masks_batch.append(attn_masks_list)
        confidences_batch.append(confs)
        labels_batch.append(label.squeeze(0))  # remove batch dim if exists

    # Pad labels to the same length
    labels_batch = pad_sequence(labels_batch, batch_first=True, padding_value=model.tokenizer.pad_token_id)

    return input_ids_list_batch, attn_masks_batch, confidences_batch, labels_batch

In [34]:
dataset = FusionDataset(
    all_outputs=[
        [
            "The capital of Japan is Tokyo.",
            "Tokyo is the capital of Japan.",
            "Japan’s capital city is Mumbai."
        ],
        [
            "The Eiffel Tower is in Paris.",
            "Paris hosts the Eiffel Tower.",
            "Eiffel Tower is located in Berlin."
        ],
        [
            "Water boils at 100 degrees Celsius.",
            "The boiling point of water is 100°C.",
            "Water freezes at 100 degrees."
        ],
        [
            "The Great Wall of China is visible from space.",
            "It is said that the Great Wall can be seen from orbit.",
            "The Great Wall of China is in India."
        ],
        [
            "The sun rises in the east.",
            "Sunset happens in the west.",
            "The sun rises in the west."
        ],
        [
            "Albert Einstein developed the theory of relativity.",
            "Einstein is known for the theory of general relativity.",
            "Newton developed the theory of relativity."
        ]
    ],
    all_confidences=[
        [0.7, 0.6, 0.2],
        [0.8, 0.75, 0.1],
        [0.9, 0.85, 0.2],
        [0.6, 0.5, 0.1],
        [0.9, 0.85, 0.3],
        [0.95, 0.9, 0.2]
    ],
    all_targets=[
        "The capital of Japan is Tokyo.",
        "The Eiffel Tower is located in Paris.",
        "Water boils at 100 degrees Celsius.",
        "The Great Wall of China is visible from space.",
        "The sun rises in the east.",
        "Albert Einstein developed the theory of relativity."
    ],
    tokenizer=model.tokenizer
)
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=fusion_collate_fn)

In [37]:
num_epochs = 1
model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = 0

    for batch in tqdm(dataloader):
        input_ids_list_batch, attn_masks_batch, confidences_batch, labels_batch = batch

        # Process each sample in the batch (since fusion model is not batchified)
        for input_ids_list, attention_masks, raw_confidences, label in zip(input_ids_list_batch, attn_masks_batch, confidences_batch, labels_batch):
            optimizer.zero_grad()

            # Move tensors to device
            input_ids_list = [x.to(device) for x in input_ids_list]
            attention_masks = [x.to(device) for x in attention_masks]
            raw_confidences = [float(c) for c in raw_confidences]
            # labels = label.unsqueeze(0).to(device)

            # decoder_input_ids = model.tokenizer("summarize:", return_tensors="pt").input_ids.to(device)

            decoder_input = "summarize:"
            decoder_input_ids = model.tokenizer(decoder_input, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)

            # Tokenize the target (label) properly
            if isinstance(label, str):
                labels = model.tokenizer(label, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
            else:
                labels = label.unsqueeze(0).to(device)

            output = model(input_ids_list, attention_masks, raw_confidences, decoder_input_ids, labels=labels)
            loss = output.loss
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

    print(f"Epoch Loss: {epoch_loss:.4f}")

Epoch 1/1


  0%|          | 0/3 [00:00<?, ?it/s]


ValueError: Expected input batch_size (3) to match target batch_size (11).

In [79]:
class ConfidenceAwareFusionModel2(nn.Module):
    def __init__(self, encoder_decoder_model_name="t5-small", sentence_encoder_name="all-mpnet-base-v2"):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(encoder_decoder_model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(encoder_decoder_model_name)
        self.sentence_encoder = SentenceTransformer(sentence_encoder_name)

        hidden_size = self.model.config.d_model
        self.conf_proj = nn.Linear(1, hidden_size)
        self.bias_mlp = nn.Sequential(
            nn.Linear(1, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )

        # Store original forward methods
        self.original_decoder_forward = self.model.decoder.forward
        self.model.decoder.forward = self.decoder_forward_with_confidence_bias

    def decoder_forward_with_confidence_bias(self,
                                          input_ids=None,
                                          attention_mask=None,
                                          encoder_hidden_states=None,
                                          encoder_attention_mask=None,
                                          inputs_embeds=None,
                                          past_key_values=None,
                                          **kwargs):

        # First get the standard decoder outputs
        decoder_outputs = self.original_decoder_forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            inputs_embeds=inputs_embeds,
            past_key_values=past_key_values,
            **kwargs
        )

        # Get the confidence scores from the encoder
        if not hasattr(self, 'current_confidence_scores') or self.current_confidence_scores is None:
            return decoder_outputs

        batch_size = decoder_outputs.last_hidden_state.size(0)

        # Modify cross-attention in each layer
        for layer in self.model.decoder.block:
            cross_attn = layer.layer[1].EncDecAttention

            # Get query states
            query_states = cross_attn.q(decoder_outputs.last_hidden_state)

            # Compute confidence bias
            # Expand confidence scores to match batch and sequence dimensions
            conf_scores = self.current_confidence_scores.to(query_states.device)
            conf_scores = conf_scores.view(1, 1, -1).expand(batch_size, query_states.size(1), -1)  # (batch, tgt_len, src_len)

            # Project confidence scores
            conf_projected = self.conf_proj(conf_scores.unsqueeze(-1)).squeeze(-1)  # (batch, tgt_len, src_len, d_model)

            # Compute dot product between query and confidence projection
            beta = torch.matmul(query_states.unsqueeze(2), conf_projected.transpose(-1, -2)).squeeze(2)  # (batch, tgt_len, src_len)

            # Apply MLP to get bias term
            bias = self.bias_mlp(beta.unsqueeze(-1)).squeeze(-1)  # (batch, tgt_len, src_len)

            # Modify attention weights
            if hasattr(cross_attn, 'attention_scores'):
                cross_attn.attention_scores = cross_attn.attention_scores + bias

        return decoder_outputs

    # Rest of your methods remain the same...
    def compute_semantic_agreement(self, answers: List[str]) -> torch.Tensor:
        embeddings = self.sentence_encoder.encode(answers, convert_to_tensor=True)
        sim_matrix = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)
        return sim_matrix

    def compute_final_confidence_scores(self, raw_confidences: List[float], sem_sim_matrix: torch.Tensor, lambda_param=0.7) -> torch.Tensor:
        n = len(raw_confidences)
        norm_conf = torch.tensor(raw_confidences) / max(raw_confidences)
        final_scores = torch.zeros_like(norm_conf)

        for i in range(n):
            sem_agree = (torch.sum(sem_sim_matrix[i]) - 1) / (n - 1)
            final_scores[i] = lambda_param * norm_conf[i] + (1 - lambda_param) * sem_agree

        return final_scores

    def encode_with_confidence(self, input_ids_list, attention_masks, scores):
        self.current_confidence_scores = scores

        all_embeddings, all_masks = [], []
        for input_ids, attn_mask, ai in zip(input_ids_list, attention_masks, scores):
            token_embeddings = self.model.encoder.embed_tokens(input_ids)
            conf_vec = self.conf_proj(torch.tensor([[ai]], device=token_embeddings.device))
            conf_vec = conf_vec.expand_as(token_embeddings)
            fused = token_embeddings + conf_vec
            all_embeddings.append(fused)
            all_masks.append(attn_mask)

        return torch.cat(all_embeddings, dim=1), torch.cat(all_masks, dim=1)

    def forward(self, input_ids_list, attention_masks, raw_confidences, decoder_input_ids, labels=None):
        answers = [self.tokenizer.decode(ids[0], skip_special_tokens=True) for ids in input_ids_list]
        sem_sim = self.compute_semantic_agreement(answers)
        final_scores = self.compute_final_confidence_scores(raw_confidences, sem_sim)

        fused_embeddings, fused_mask = self.encode_with_confidence(input_ids_list, attention_masks, final_scores)

        output = self.model(
            inputs_embeds=fused_embeddings,
            attention_mask=fused_mask,
            decoder_input_ids=decoder_input_ids,
            labels=labels,
            return_dict=True
        )

        if hasattr(self, 'current_confidence_scores'):
            del self.current_confidence_scores

        return output

    def generate(self, input_ids_list, attention_masks, raw_confidences, max_length=128):
        answers = [self.tokenizer.decode(ids[0], skip_special_tokens=True) for ids in input_ids_list]
        sem_sim = self.compute_semantic_agreement(answers)
        final_scores = self.compute_final_confidence_scores(raw_confidences, sem_sim)

        fused_embeddings, fused_mask = self.encode_with_confidence(input_ids_list, attention_masks, final_scores)

        generated_ids = self.model.generate(
            inputs_embeds=fused_embeddings,
            attention_mask=fused_mask,
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

        if hasattr(self, 'current_confidence_scores'):
            del self.current_confidence_scores

        return self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [80]:
fusion_model2 = ConfidenceAwareFusionModel2().to(device)

In [81]:
fusion_model2

ConfidenceAwareFusionModel2(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Li

In [86]:
fusion_model2 = ConfidenceAwareFusionModel2().to(device)

# Example inputs
llm_outputs = [
    "The capital of Japan is Tokyo.",
    "Tokyo is the capital of Japan.",
    "Japan's capital city is Mumbai.",
     "I dont know"
]
confidences = [0.7, 0.6, 0.2,0.6]

# Tokenize all answers
input_ids_list, attn_masks = [], []
for out in llm_outputs:
    encoded = fusion_model2.tokenizer(out, return_tensors="pt", padding=True, truncation=True).to(device)
    input_ids_list.append(encoded['input_ids'])
    attn_masks.append(encoded['attention_mask'])

# For training
target = "The capital of Japan is Tokyo."
labels = fusion_model2.tokenizer(target, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)

# Training call (note: decoder_input_ids is handled internally by T5)
output = fusion_model2(
    input_ids_list=input_ids_list,
    attention_masks=attn_masks,
    raw_confidences=confidences,
    decoder_input_ids=None,  # Let T5 handle this
    labels=labels
)
print("Training loss:", output.loss.item())

# For inference/generation
fused_answer = fusion_model2.generate(
    input_ids_list=input_ids_list,
    attention_masks=attn_masks,
    raw_confidences=confidences,
    max_length=128  # You can adjust this
)
print("Fused Answer:", fused_answer)

Training loss: 0.9402994513511658
Fused Answer: Die Hauptstadt Japans ist Tokyo.
